# Trabajo Practico 1 - Analisis de Series Temporales

## Importacion de Librerias

In [7]:
import pandas as pd

import numpy as np

import math

import seaborn as sns

import scipy.stats
from scipy import stats

import pylab

import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection

import statsmodels.tsa.stattools as tsa
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import grangercausalitytests

from pmdarima.arima.utils import ndiffs
from pmdarima import auto_arima

from sklearn.metrics import mean_absolute_error, mean_squared_error
from arch.unitroot import PhillipsPerron

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
tema_azul = ['#0000CC','#3366FF','#99CCFF']
tema_rojo = ['#CC0000','#FF3333','#FF9999']
tema_verde = ['#009900','#33AA33','#55BB55']
tema_negro = ['#000000','#333333','#555555']

## Definicion de Funciones

### Trabajo de Datos

In [10]:
def filtrar_serie(serie, start_date, end_date):
    serie = serie[(serie.index >= start_date) & (serie.index <= end_date)]
    return serie

### Graficas

In [12]:
def autocov_autocorr(s1,s2, numlag= 20, color = ['#000000','#333333','#555555']):
  p1, axes = plt.subplots(3,2,figsize=(14,9), sharex=True, sharey=False )

  axes[2,0].set_xlabel('Lags', fontsize=8)
  axes[2,1].set_xlabel('Lags', fontsize=8)
  
  axes[0,0].set_ylabel('FAC', fontsize=8)
  axes[1,0].set_ylabel('FACP', fontsize=8)
  axes[2,0].set_ylabel('FAS', fontsize=8)

  #grafico correlacion
  plot_acf(s1, lags=numlag, ax= axes[0,0], color=color[0], label="ACF", vlines_kwargs={"colors":color[0]})
  axes[0,0].set_title(label = s1.name, fontsize=9)
  axes[0,0].tick_params(axis='both', labelsize=7)
  for item in axes[0,0].collections:
    #change the color of the CI 
    if type(item)==PolyCollection:
        item.set_facecolor('#AAAAAA')

  plot_acf(s2, lags=numlag, ax= axes[0,1], color=color[0], label="ACF", vlines_kwargs={"colors":color[0]}, title=s2.name)
  axes[0,1].set_title(label = s2.name, fontsize=9)
  axes[0,1].tick_params(axis='both', labelsize=7)
  for item in axes[0,1].collections:
    #change the color of the CI 
    if type(item)==PolyCollection:
        item.set_facecolor('#AAAAAA')
  
  #grafico correlacion parcial
  plot_pacf(s1, lags=numlag, ax= axes[1,0], color=color[1], label="PACF", vlines_kwargs={"colors":color[1]}, title='', method='ywm')
  axes[1,0].tick_params(axis='both', labelsize=7)
  for item in axes[1,0].collections:
    #change the color of the CI 
    if type(item)==PolyCollection:
        item.set_facecolor('#AAAAAA')

  plot_pacf(s2, lags=numlag, ax= axes[1,1], color=color[1], label="PACF", vlines_kwargs={"colors":color[2]}, title='', method='ywm')
  axes[1,1].tick_params(axis='both', labelsize=7)
  for item in axes[1,1].collections:
    #change the color of the CI 
    if type(item)==PolyCollection:
        item.set_facecolor('#AAAAAA')


  #grafico covarianza
  axes[2,0].plot(tsa.acovf(s1,fft=False, nlag=numlag), color=color[2], label='AutoCov')
  axes[2,0].tick_params(axis='both', labelsize=7)
  axes[2,1].plot(tsa.acovf(s2,fft=False, nlag=numlag), color=color[2], label='AutoCov')
  axes[2,1].tick_params(axis='both', labelsize=7)

  #p1.legend()
  plt.show()
#autocov_autocorr(df.ventas_ajustado, df.diff1, 20, tema_rojo)

In [13]:
def prueba_residuos(resid, color = ['#000000','#333333','#555555']):
    p1, axes = plt.subplots(3,2,figsize=(15,10))
    #p1, axes = plt.subplot_mosaic('AB;CD;EF;GG')

    plot_acf(resid, ax= axes[0,0], color=color[0], vlines_kwargs={"colors":color[0]})
    axes[0,0].set_title(label = 'Autocorrelacion', fontsize=10)
    axes[0,0].tick_params(axis='both', labelsize=7)
    axes[0,0].get_lines()[1].set(markersize = 5.0, markerfacecolor = color[1], markeredgecolor = color[2])
    for item in axes[0,0].collections:
        #change the color of the CI 
        if type(item)==PolyCollection:
            item.set_facecolor('#AAAAAA')

    #plot_pacf(resid, ax= axes[0,1]);
    plot_pacf(resid, ax= axes[0,1], color=color[0], vlines_kwargs={"colors":color[0]})
    axes[0,1].set_title(label = 'Autocorrelacion Parcial', fontsize=10)
    axes[0,1].tick_params(axis='both', labelsize=7)
    axes[0,1].get_lines()[1].set(markersize = 5.0, markerfacecolor = color[1], markeredgecolor = color[2])
    for item in axes[0,1].collections:
        #change the color of the CI 
        if type(item)==PolyCollection:
            item.set_facecolor('#AAAAAA')

    axes[1,0].plot(resid, color = color[1])
    axes[1,0].set_title("Residuos del modelo",size=10)
    axes[1,0].tick_params(axis='both', labelsize=7)

    sns.distplot(resid,bins=12,color=color[1], ax=axes[1,1])
    axes[1,1].set_title(f'Histograma de residuos - Shapiro p-value = {stats.shapiro(resid).pvalue:.4f}',size=10)

    scipy.stats.probplot(resid, plot = axes[2,0])
    axes[2,0].set_title("QQ Plot", size = 10)
    axes[2,0].get_lines()[0].set(markersize = 3.0, markerfacecolor = color[1], markeredgecolor = color[2], color = color[1])
    axes[2,0].get_lines()[1].set(color = color[1])
    axes[2,0].tick_params(axis='both', labelsize=7)

    Incorr_residuos_modelo=sm.stats.acorr_ljungbox(resid, lags=10)
    axes[2,1].plot(Incorr_residuos_modelo.lb_pvalue, marker='o', markersize=10, linestyle='--', linewidth=1)
    axes[2,1].set_title("Grafico de incorrelacion de los residuos del modelo", size = 10)
    axes[2,1].set_ylabel("P-valores ")
    axes[2,1].get_lines()[0].set(markersize = 10.0, markerfacecolor = color[1], markeredgecolor = color[2], color = color[1])
    axes[2,1].tick_params(axis='both', labelsize=7)

#prueba_residuos(modelo_sarima.resid, tema_verde)

In [14]:
def graficar_pred(train, test, pred, confint, xlabel="Fecha", ylabel="ARS$", color = ['#000000','#333333','#555555']):
    p1, axes = plt.subplot_mosaic('AB;CC',figsize=(15,10))

    axes['A'].plot(train, color = color[0])
    axes['A'].set_xlabel(xlabel)
    axes['A'].set_ylabel(ylabel)
    axes['A'].set_title('Serie Original')
    axes['A'].grid(True, color='0.6', dashes=(5,2,1,2))

    axes['B'].plot(confint.iloc[:,1],label="", color='#222222')
    axes['B'].plot(confint.iloc[:,0],label="", color='#222222')
    axes['B'].plot(test, label="Valores Observados", color = color[0])
    axes['B'].plot(pred, label="Predicciones", color = color[2])
    axes['B'].fill_between(pred.index, confint.iloc[:,1], confint.iloc[:,0], facecolor=color[2], alpha=0.3)
    axes['B'].fill_between(pred.index, pred, test, facecolor=color[1], alpha=0.6)
    axes['B'].set_xlabel(xlabel)
    axes['B'].set_ylabel(ylabel)
    axes['B'].set_title('Predicciones - Intervalo de Confianza')
    axes['B'].legend(loc=0)
    axes['B'].grid(True, color='0.6', dashes=(5,2,1,2))

    axes['C'].plot(train, color = color[0])
    axes['C'].plot(test, color = color[1])
    axes['C'].plot(pred, color = color[2])
    axes['C'].set_xlabel(xlabel)
    axes['C'].set_ylabel(ylabel)
    axes['C'].set_title('Valores Reales vs Observados')
    axes['C'].fill_between(pred.index, pred, test, facecolor=color[1], alpha=0.8)
    axes['C'].fill_between(pred.index, confint.iloc[:,1], confint.iloc[:,0], facecolor=color[2], alpha=0.3)
    axes['C'].grid(True, color='0.6', dashes=(5,2,1,2))

#graficar_pred(train, test[:16], predicciones_sarima.predicted_mean, predicciones_sarima.conf_int(), color = tema_azul)

### Tests Estadisticos

In [15]:
def print_test_adf(y):
  print("_".center(120, '_'))
  print(f'Augmented Dickey-Fuller')
  print(f'Estadistico ADF\t\tp-Valor\tEstacionaridad\tModo')
  resultado = adfuller(y, regression='c')
  print(f'{resultado[0]:.4f} \t\t{resultado[1]:.4f} \t{"No " if resultado[1] > 0.05 else "Si"}\t\tConstante sola')

  resultado = adfuller(y, regression='ct')
  print(f'{resultado[0]:.4f} \t\t{resultado[1]:.4f} \t{"No " if resultado[1] > 0.05 else "Si"}\t\tConstante y Tendencia Lineal')

  resultado = adfuller(y, regression='ctt')
  print(f'{resultado[0]:.4f} \t\t{resultado[1]:.4f} \t{"No " if resultado[1] > 0.05 else "Si"}\t\tConstante y Tendencia Lineal y Cuadratica')

  resultado = adfuller(y, regression='n')
  print(f'{resultado[0]:.4f} \t\t{resultado[1]:.4f} \t{"No " if resultado[1] > 0.05 else "Si"}\t\tSin Contante ni Tendencia')
  print("_".center(120, '_'))
#print_test_adf(df.ventas_ajustado)
#print_test_adf(df.diff1)

In [16]:
def Phillips_Perron(series, **k):

    params = {'n':'No incluye término independiente ni lineal',
              'c':'Con término independiente, Sin término lineal',
              'ct':'Incluye ambos términos'
              }
    print("_".center(120, '_'))
    print(f'Phillips-Perron')
    print(f'Estadistico PP\tp-Valor\t\tNumLags\t\tEstacionaridad\tnDiffs\tTipo_Regresion')
    for param in params:
        pp = PhillipsPerron(series,trend=param, **k)
        print(f"{pp.stat:.2f}\t\t{pp.pvalue:.4f}\t\t{pp.lags}\t\t{'No' if pp.pvalue > 0.05 else 'Si'}\t\t{ndiffs(series, test='pp')}\t{param}-{params.get(param)}")
    print("_".center(120, '_'))

#Phillips_Perron(df.ventas_ajustado)
#Phillips_Perron(df.diff1)

In [17]:
def kpss_test(series, **kw):

    params = {'c':'estacionarios alrededor de una constante.',
              'ct': 'estacionarios alrededor de una tendencia.'
             }
    print("_".center(120, '_'))
    print(f'KPSS')
    print(f'Estad. KPSS\tp-Valor\t\tNumLags\t\tEstacionaridad\tnDiffs\tTipo_Regresion')
    
    for param in params:
        statistic, p_value, n_lags, critical_values = kpss(series,regression = param, **kw)
        print(f"{statistic:.4f}\t\t{p_value:.4f}\t\t{n_lags}\t\t{'No' if p_value < 0.05 else 'Si'}\t\t{ndiffs(series, test='kpss')}\t{param} - {params.get(param)}")
    print("_".center(120, '_'))
#kpss_test(df.ventas_ajustado)

In [18]:
def medidas_error(serie_test, serie_pred):
    MSE = mean_squared_error(serie_test, serie_pred)
    MAE = mean_absolute_error(serie_test, serie_pred)
    RMSE = np.sqrt(mean_squared_error(serie_test, serie_pred))
    MAPE = np.mean(abs((serie_test-serie_pred)/serie_test))

    return MSE, MAE, RMSE, MAPE
#evaluar_metricas(test[:16], predicciones_sarima.predicted_mean )

In [19]:
def evaluar_metricas(serie_test, serie_pred):
    print(f'MSE:\t{mean_squared_error(serie_test, serie_pred):.0f}')
    print(f'MAE:\t{mean_absolute_error(serie_test, serie_pred):.0f}')
    print(f'RMSE:\t{np.sqrt(mean_squared_error(serie_test, serie_pred)):.0f}')
    print(f'MAPE:\t{round(np.mean(abs((serie_test-serie_pred)/serie_test)),4):.3f}')

#evaluar_metricas(test[:16], predicciones_sarima.predicted_mean )

In [20]:
def grangers_causation_matrix(data, variables, maxlag, testgr='ssr_chi2test', verbose=False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [(test_result[i+1][0][testgr][1],i+1) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

#grangers_causation_matrix(train2, variables = train2.columns, maxlag=5, verbose=False)

In [21]:
def highlight_min(s):
    is_min = s == s.min()
    return [f'background-color: green' if v else '' for v in is_min]

In [22]:
def calcular_errores(test_serie, modelos_alt_nombres, modelos_alt, per = 16):
    df_errores = pd.DataFrame()
    df_errores['Modelo'] = modelos_alt_nombres
    df_errores['Akaike'] = [m.aic for m in modelos_alt]

    i, MSE, MAE, RMSE, MAPE = 0, [], [], [], []
    for m in modelos_alt:
        pred_m = m.get_forecast(per)
        m_MSE, m_MAE, m_RMSE, m_MAPE = medidas_error(test_serie[:per], pred_m.predicted_mean )
        MSE = np.append(MSE, m_MSE)
        MAE = np.append(MAE, m_MAE)
        RMSE = np.append(RMSE, m_RMSE)
        MAPE = np.append(MAPE, m_MAPE)

    df_errores['MSE'] = MSE
    df_errores['MAE'] = MAE
    df_errores['RMSE'] = RMSE
    df_errores['MAPE'] = MAPE
    
    df_errores.style.apply(highlight_min, subset=['Akaike', 'MSE', 'MAE','RMSE','MAPE'])\
        .format({'Akaike': "{:.0f}",'MSE': "{:.2E}",'MAE': "{:.2E}",'RMSE': "{:.2E}",'MAPE': "{:.0%}"})
    
    return df_errores

#df_err = calcular_errores(test, modelos_alt_nombres, modelos_alt, 10)


## Importacion y Trabajo de Datos

In [38]:
vent_full = pd.read_excel('.\Data\Descarga_VENTAS_POR_LOCALES_NB_050923.xlsx',skiprows=1)
# df_vent = vent_full.groupby(by = 'Periodo', sort = True)['Importe'].sum().reset_index()
# df_vent.rename(columns={'Periodo': 'mes', 'Importe': 'ventas'}, inplace = True)
# df_vent.mes = pd.to_datetime(df_vent.mes, dayfirst = True, format = '%m/%Y')
# df_vent = df_vent.sort_values(by = 'mes')
# df_vent.set_index('mes', inplace = True)
# Drop column: 'Valores'
vent_full = vent_full.drop(columns=['Valores'])
# Drop column: 'Centro Comercial'
vent_full = vent_full.drop(columns=['Centro Comercial'])
# Drop column: 'Razón Social'
vent_full = vent_full.drop(columns=['Razón Social'])
# Drop column: 'Identificacion'
vent_full = vent_full.drop(columns=['Identificacion'])
# Drop column: 'Contrato'
vent_full = vent_full.drop(columns=['Contrato'])
# Drop column: 'Estado'
vent_full = vent_full.drop(columns=['Estado'])
# Drop column: 'Faltan'
vent_full = vent_full.drop(columns=['Faltan'])
# Drop column: 'Cero'
vent_full = vent_full.drop(columns=['Cero'])
# Drop column: 'Calculo'
vent_full = vent_full.drop(columns=['Calculo'])
# Change column type to category for column: 'Nombre Comercial'
vent_full = vent_full.astype({'Nombre Comercial': 'category'})
# Change column type to category for column: 'Local'
vent_full = vent_full.astype({'Local': 'category'})
# Change column type to category for column: 'Rubro'
vent_full = vent_full.astype({'Rubro': 'category'})
# Change column type to datetime64[ns] for column: 'Periodo'
vent_full = vent_full.astype({'Periodo': 'datetime64[ns]'})
# Change column type to category for column: 'Rubros'
vent_full = vent_full.astype({'Rubros': 'category'})

In [94]:
df_ipc = pd.read_excel('.\Data\Tabla_IPC.xlsx',skiprows=1)
df_ipc.rename(columns={'Fecha': 'fecha', 'Indice': 'ipc'}, inplace = True)
df_ipc.set_index('fecha', inplace=True)
df_ipc['ipc_previo'] = df_ipc['ipc'].shift(1)
df_ipc['ipc_intermensual'] = (df_ipc['ipc'] - df_ipc['ipc_previo']) / df_ipc['ipc']
df_ipc.reset_index(inplace=True)
df_ipc.head()

,fecha,ipc,ipc_previo,ipc_intermensual
0,2016-12-01,100.0000,NaN,NaN
1,2017-01-01,101.5859,100.0000,0.015611
2,2017-02-01,103.6859,101.5859,0.020253
3,2017-03-01,106.1476,103.6859,0.023191
4,2017-04-01,108.9667,106.1476,0.025871


In [40]:
df_usd = pd.read_excel('.\Data\Dolar_Evolución.xlsx',skiprows=0)
df_usd = df_usd.iloc[:, 1:]
df_usd.columns = ['fecha', 'ofi', 'blue']
df_usd.set_index('fecha', inplace=True)
df_usd.head()

,ofi,blue
fecha,,
2015-01-01,8.55,13.50
2015-02-01,8.63,12.93
2015-03-01,8.72,12.52
2015-04-01,8.81,12.60
2015-05-01,8.90,12.55


In [183]:
df_clima = pd.read_excel('.\Data\Clima.xlsx',skiprows=0)

Proceso de datos

In [170]:
df_vent = vent_full
df_vent = pd.merge(df_vent, df_ipc, left_on = 'Periodo', right_on= 'fecha' , how='outer')
df_vent = pd.merge(df_vent, df_usd, left_on = 'Periodo', right_on= 'fecha' , how='left')
df_vent.dropna(subset=['ipc'], inplace=True)
df_vent.dropna(subset=['blue'], inplace=True)
df_vent['ventas_ajustado'] = df_vent['Importe'] / (1 + df_vent['ipc_intermensual']) / df_vent['blue']

In [171]:
df_vent = df_vent.sort_values(by = ['Nombre Comercial', 'Periodo'])

for i in range(12):
    df_vent[f'lag{i+1}'] = df_vent.groupby(['Nombre Comercial'])['ventas_ajustado'].shift(periods=i+1)

df_vent['diff1'] = df_vent.groupby(['Nombre Comercial'])['ventas_ajustado'].diff()

In [178]:
df_vent['quarter'] = df_vent['Periodo'].dt.quarter
df_vent['month'] = df_vent['Periodo'].dt.month
df_vent['year'] = df_vent['Periodo'].dt.year

In [189]:
df_vent = pd.merge(df_vent, df_clima, left_on='Periodo', right_on='fecha', how='left')